### Build a Simple LLM Application with LCEL
Build a simple LLM application with LangChain. This application will translate text from English into another language. It's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain.




- Used LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing the application using LangSmith

- Deploying application with LangServe

In [1]:
!pip install langchain

  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.3.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.120-py3-none-any.whl.metadata (13 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached orjson-3.10.7-cp310-none-win_amd64.whl.metadata (51 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
Using cached langchain-0.3.0-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.0-py3-none-any.whl (405 kB)
Using cached langchain_text_splitters-0.3.0-py3-none-any.whl (25 kB)
Using cached langsmith-0.1.120-py3-none-any.whl (289 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached orjson-3.10.7-cp310-none-win_amd64.whl (137 kB)
Using cached jsonpointer-3

In [8]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

#import openai
#openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [9]:
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [3]:
!pip install langchain_groq

In [10]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000212BD0FA8C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000212BCBE3F10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
!pip install langchain_core

In [12]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [13]:
result

AIMessage(content="Here's the translation, with a couple of options:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous?** \n\n**Informal:**\n\n* **Salut, comment vas-tu?** \n\n\nLet me know if you'd like more variations!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.000144079, 'queue_time': 0.013215128999999999, 'total_time': 0.107416806}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0417854b-e82f-4009-b50f-2f872212b429-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80})

In [14]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

"Here's the translation, with a couple of options:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous?** \n\n**Informal:**\n\n* **Salut, comment vas-tu?** \n\n\nLet me know if you'd like more variations!\n"

In [15]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here\'s the translation, along with some nuances:\n\n**Formal:**\n\n* **Bonjour. Comment allez-vous ?** (pronounced: bohn-zhoor. koh-mahn ah-lay-voo)\n\n**Informal:**\n\n* **Salut. Comment vas-tu ?** (pronounced: sa-loo. koh-mahn vah-too)\n\n**Important Notes:**\n\n* **Formal vs. Informal:**  French has distinct formal and informal ways of addressing people. Use "vous" (you) for formal situations, and "tu" (you) for informal situations with friends, family, or people your own age.\n* **"Salut"** is very casual, similar to "Hi" or "Hey" in English.\n\n\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [16]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [17]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [18]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [19]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [25]:
!pip install streamlit